# Importación de librerías

In [36]:
import numpy as np
import pandas as pd
import os
import csv
import random
import _pickle as pickle
from PIL import Image
from skimage.transform import resize

from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras import layers

import matplotlib.pyplot as plt

# Carga de Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Lectura de los datos

In [4]:
# Carga de los dataframes de train, test y validation
df_train = pd.read_csv("/content/drive/MyDrive/TFM/train_labels.csv")
df_test = pd.read_csv("/content/drive/MyDrive/TFM/test_labels.csv")
df_validation = pd.read_csv("/content/drive/MyDrive/TFM/validation_labels.csv")

print("El número de registros en df_train es: " + str(len(df_train)))
print("El número de registros en df_test es: " + str(len(df_test)))
print("El número de registros en df_validation es: " + str(len(df_validation)))

El número de registros en df_train es: 32072
El número de registros en df_test es: 3000
El número de registros en df_validation es: 3000


In [27]:
df_train.head()

,PatientId,Target
0,ba8f8cb0-5aed-4242-b117-8516f4c7b51b,0
1,49f5c9f1-8c3c-4afc-b940-33e1500d31e8,0
2,0cc27255-7f78-4093-84d7-04190a7d72ec,0
3,a12b6a40-323c-4ead-8875-5b84ae4beb6f,0
4,de94813d-0475-41b0-b248-af03dfb5a412,0


In [6]:
df_test.head()

,PatientId,Target
0,a74b7b44-ae2b-48cc-bc0a-597d221a4d08,0
1,da0483cc-6cd4-40c5-aaef-b085a01cd76c,0
2,7658ef50-d386-4066-92d9-5d5735af5f3c,0
3,922fa0c4-f867-4102-9741-5952a84c3fee,0
4,cab1fcd3-7e67-4928-8612-80bae540ca2c,0


In [7]:
df_validation.head()

,PatientId,Target
0,6a2d8b64-cf13-47e2-bed9-41a6b801e1fb,1
1,97a6f7c7-df3f-4127-907d-27823c48f2fe,1
2,b434200b-248e-4aa3-aeff-b0c9351a65fc,1
3,9abdc28f-151f-4243-8595-1751a8ad1286,0
4,bf88dd53-1b28-44b4-b101-33b49816d8a0,0


## Comprobación de los datos

Vamos a comprobar que las imágenes que se encuentran en cada directorio, es decir, en train, validation, y test, también están en los dataframes correspondientes.

Se hace esta comprobación básicamente para saber si se han subido de forma correcta las imágenes.

In [23]:
def check_images(dir, df):
  files = os.listdir(dir)
  num_elements = len(df)
  num_images = len(files)

  # Recorremos el dataframe correspondiente y comprobamos si están las imágenes
  for patientId in df["PatientId"]:
    if not patientId + ".png" in files:
      return False

  # Comprobamoos que hay el mismo número de registros que de imágenes
  if num_elements != num_images:
    return False

  return True

In [25]:
print("Train: " + str(check_images("/content/drive/MyDrive/TFM/train_images", df_train)))
print("Test: " + str(check_images("/content/drive/MyDrive/TFM/test_images", df_test)))
print("Validation: " + str(check_images("/content/drive/MyDrive/TFM/validation_images", df_validation)))

Train: True
Test: True
Validation: True


# Carga de las imágenes preprocesadas

En este punto vamos a cargar todas las imágenes que tenemos en un diccionario, es decir, tanto las imágenes de train, como de test y validation.

Este diccionario está formado por un par clave-valor, donde la clave va a ser el `PatientId` (el nombre de la imágenes sin el ".png"), y el valor va a ser la imágenes preprocesada en formato 224x224 (neceario para hacer uso de EfficientNetB0).

Se ha hecho uso de esta carga en un diccionario para reducir los tiempos de ejecución en el entrenamiento de la red, ya que anteriormente el Data Generator tardaba demasiado por dos motivos:
* Al abrir la imagen en cada época.
* Al redimensionar la imagen del formato original 1024x1024 a 224x224.

In [28]:
# EXTRA
TRAIN_FOLDER = "/content/drive/MyDrive/TFM/train_images"
TEST_FOLDER = "/content/drive/MyDrive/TFM/test_images"
VALID_FOLDER = "/content/drive/MyDrive/TFM/validation_images"

In [45]:
def get_dic_images(dir, dic_images):
  files = os.listdir(dir)
  dic = dic_images.copy()

  for filename in files:
    patientId = filename.split(".")[0]
    image = np.array(Image.open(os.path.join(dir, filename)))
    dic[patientId] = image
  
  return dic

In [46]:
dic_images = {}
dic_images = get_dic_images(TRAIN_FOLDER, dic_images)
dic_images = get_dic_images(TEST_FOLDER, dic_images)
dic_images = get_dic_images(VALID_FOLDER, dic_images)

In [47]:
# Volcamos los datos a un fichero
pickle.dump(dic_images, open(os.path.join("/content/drive/MyDrive/TFM", "dic_images.pickle"), 'wb'))

Para no tener que cargar todo el rato el diccionario, ya que esto es muy costoso (estar abriendo cada imagen), hemos el diccionario a un fichero pickle.

De esta forma, conseguimos reducir el tiempo de carga de las imágenes. Finalmente, definimos la carga en el caso de ser necesaria: 

In [51]:
# Hacemos la lectura de los datos
dic_images = pickle.load(open(os.path.join("/content/drive/MyDrive/TFM", "dic_images.pickle"), "rb"))

# Comprobamos que se ha hecho la lectura de forma correcta
print("El número total de imágenes es: " + str(len(dic_images)))
print("El número total de imágenes esperadas es: " + str(len(df_train) + len(df_test) + len(df_validation)))

El número total de imágenes es: 38072
El número total de imágenes esperadas es: 38072


# Data Generator

En este punto vamos a resolver la problemática de la carga de los datos en memoria. El dataset de entrenamiento presenta 32072 imágenes con resolución 224x224, lo cual hace que haya un elevado consumo de memoria, hasta tal punto que puede que Google Colab no sea capaz de soportar.

Para solventar este problema creamos un data generator, el cual se encarga de cargar en memoria pequeños grupos de imágenes según se vayan utilizando, es decir, dependiendo del tamaño del batch.

La salida que proporciona el data generator es un cojunto de imágenes junto con la variable objetivo.

Cabe destacar que al usar EfficientNet la imagen de entrada a la red tiene que tener valores entre 0 y 255, es por ello que no se normalizan los datos porque EfficientNet ya tiene sus layers para hacerlo.


In [69]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(keras.utils.Sequence):

  # Constructor
  def __init__(self, folder, dic_images, batch_size=32, image_size=256, shuffle=True, predict=False):
    self.folder = folder
    self.filenames = os.listdir(folder)
    self.dic_images = dic_images.copy()
    self.batch_size = batch_size
    self.image_size = image_size
    self.shuffle = shuffle
    self.predict = predict
    self.on_epoch_end()
    

  # Carga y Transformación de la imágenes para training
  # filename: es el nombre del archivo de la imagen, es decir, con png
  def __train__(self, filename):
    filename = filename.split(".")[0]

    # Cargamos la imagen original
    img = self.dic_images[filename]

    # Cargamos la variable objetivo
    target = df_train[df_train["PatientId"] == filename]["Target"].item()

    # Reducción de la escala de la imagen
    if (self.image_size, self.image_size) != (IMG_SIZE, IMG_SIZE): 
      img = resize(img, (self.image_size, self.image_size), mode="reflect") * 255
      img = img.astype(np.uint8)

    # Normalizamos
    # img_min = img.min()
    # img_max = img.max()
    # img_norm = (img - img_min) / (img_max - img_min)

    # Expandimos las dimensiones (self.image_size, self.image_size, 1)
    # img_norm = np.expand_dims(img_norm, -1)
    img = np.expand_dims(img, -1)

    return img, target


  # Carga y transformación de las imágenes para testing
  def __test__(self, filename):
    filename = filename.split(".")[0]

    # Cargamos la imagen original
    img = self.dic_images[filename]

    # Reducción de la escala de la imagen
    if (self.image_size, self.image_size) != (IMG_SIZE, IMG_SIZE): 
      img = resize(img, (self.image_size, self.image_size), mode="reflect") * 255
      img = img.astype(np.uint8)

    # Normalizamos
    # img_min = img.min()
    # img_max = img.max()
    # img_norm = (img - img_min) / (img_max - img_min)

    # Expandimos las dimensiones (self.image_size, self.image_size, 1)
    # img_norm = np.expand_dims(img_norm, -1)
    img = np.expand_dims(img, -1)

    return img

  
  # Método encargado de generar el batch
  def __getitem__(self, index):
    # Generación de los nombres de archivos pertenecientes al batch
    filenames_batch = self.filenames[index*self.batch_size:(index+1)*self.batch_size]

    if self.predict:
      # Modo testing
      imgs = [self.__test__(filename) for filename in filenames_batch]
      imgs = np.array(imgs)
      return imgs, filenames_batch

    else:
      # Modo training
      items = [self.__train__(filename) for filename in filenames_batch]
      imgs, targets = zip(*items)
      imgs = np.array(imgs)
      targets = np.expand_dims(np.array(targets), -1)
      return imgs, targets

  # Método encargado de mezclar nos nombres de archivos, para así dotar de una mayor aleatoriedad
  def on_epoch_end(self):
    if self.shuffle:
      random.shuffle(self.filenames)

  # Método para controlar el tamaño del 
  def __len__(self):
    return int(len(self.filenames) / self.batch_size)

In [ ]:
df_train[df_train["PatientId"] == "0004cfab-14fd-4e49-80ba-63a80b6bddd6"]["Target"].item()

In [59]:
dg = DataGenerator(folder=TRAIN_FOLDER, batch_size=32, image_size=224)
dg.__len__()

1002

In [62]:
filenames_batch = dg.filenames[0*dg.batch_size:(0+1)*dg.batch_size]

In [67]:
import time

inicio_1 = 0
inicio_2 = 0
inicio_3 = 0
inicio_4 = 0
inicio_5 = 0
inicio_6 = 0
fin_1 = 0
fin_2 = 0
fin_3 = 0
fin_4 = 0
fin_5 = 0
fin_6 = 0



inicio_1 = time.time()

filenames_batch = dg.filenames[0*dg.batch_size:(0+1)*dg.batch_size]
items = [] #[dg.__train__(filename) for filename in filenames_batch]
for filename in filenames_batch:

  inicio_2 += time.time()
  # Cargamos la imagen original
  # img = np.array(Image.open(os.path.join(dg.folder, filename)))
  img = dic_images[filename.split(".")[0]]
  fin_2 += time.time()

  inicio_3 += time.time()
  # Cargamos la variable objetivo
  target = df_train[df_train["PatientId"] == filename.split(".")[0]]["Target"].item()
  fin_3 += time.time()

  # Reducción de la escala de la imagen
  inicio_4 += time.time()
  if (dg.image_size, dg.image_size) != (224,224): 
    img = resize(img, (dg.image_size, dg.image_size), mode="reflect")
  fin_4 += time.time()

  # Normalizamos
  # img_min = img.min()
  # img_max = img.max()
  # img_norm = (img - img_min) / (img_max - img_min)

  # Expandimos las dimensiones (self.image_size, self.image_size, 1)
  # img_norm = np.expand_dims(img_norm, -1)
  inicio_5 = time.time()
  img = np.expand_dims(img, -1)
  fin_5 = time.time()

  items.append([img, target])

fin_1 = time.time()



inicio_6 = time.time()

imgs, targets = zip(*items)
imgs = np.array(imgs)
targets = np.expand_dims(np.array(targets), -1)

fin_6 = time.time()

tramo_1 = fin_1-inicio_1
tramo_2 = fin_2-inicio_2
tramo_3 = fin_3-inicio_3
tramo_4 = fin_4-inicio_4
tramo_5 = fin_5-inicio_5
tramo_6 = fin_6-inicio_6

print("*"*50)
print("El tramo 1 dura (s): " + str(tramo_1))
print("El tramo 2 dura (s): " + str(tramo_2))
print("El tramo 3 dura (s): " + str(tramo_3))
print("El tramo 4 dura (s): " + str(tramo_4))
print("El tramo 5 dura (s): " + str(tramo_5))
print("  => Suma de subtramos (s):" +  str(tramo_2 + tramo_3 + tramo_4 + tramo_5))
print("*"*50)
print("El tramo 6 dura (s): " + str(tramo_6))


**************************************************
El tramo 1 dura (s): 0.14699792861938477
El tramo 2 dura (s): 0.0001983642578125
El tramo 3 dura (s): 0.14495086669921875
El tramo 4 dura (s): 6.103515625e-05
El tramo 5 dura (s): 2.9087066650390625e-05
  => Suma de subtramos (s):0.14523935317993164
**************************************************
El tramo 6 dura (s): 0.0009684562683105469


# Red Neuronal Convolucional

El siguiente paso es crear la red neuronal convolucional, para ello vamos a hacer uso de `EfficientNet`, en nuestro caso usaremos `EfficientNetB0`.

Al usar `EfficientNetB0` estamos "limitados" a que la resolución de las imágenes sea de 224x224, en vez de 1024x1024 que era el tamaño original.

Otro punto a destacar es que la imagen que introduzcamos al modelo no hace falta normalizarla, ya que se encarga `EfficientNetB0` de hacerlo.

## Hiperparámetros

In [68]:
# Data generator
IMG_SIZE = 224
BATCH_SIZE = 32

# CNN
NUM_CLASSES = 2
LEARNING_RATE = 1e-3
EPOCHS = 20

## Data augmentation

El siguiente punto es definir el data augmentation, para así mejorar la variabilidad del entrenamiento y por lo tanto la precisión de la red neuronal.

In [70]:
img_augmentation = Sequential(
    [
     layers.RandomRotation(factor=0.05),
     layers.RandomTranslation(height_factor=0.05, width_factor=0.02),
     layers.RandomFlip("horizontal"),
     layers.RandomContrast(factor=0.05),
    ],
    name = "img_augmentation"
)

## CNN y transfer learning

El siguiente paso es crear la red neuronal convolucional y aplicar transefer learning.

El transfer learning lo que significa es que vamos a tener dos modelos, un modelo base con pesos ya pre-entrenados con otro tipo de problemas, y un modelo nuevo basado en el modelo base.

De forma resumida lo que se va a hacer es lo siguiente:
* 1º: vamos a inicializar el modelo base con pesos ya pre-entrenados.
* 2º: se van a congelar todos los layers del modelo base con `trainable = False`.
* 3º: Creamos un nuevo modelo después de la salida del modelo base.
* 4º: Entrenamos el nuevo modeloo con el dataset.

In [71]:
# Método que se encarga de generar el modelo
# https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/#transfer-learning-from-pretrained-weights
# https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images
def build_model():
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
  x = img_augmentation(inputs)
  x = layers.Concatenate()([x, x, x])  
  model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

  # Freeze the pretrained weights
  model.trainable = False

  # Rebuild top
  x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
  x = layers.BatchNormalization()(x)

  top_dropout_rate = 0.2
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  # outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)
  outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

  # Compile
  model = keras.Model(inputs, outputs, name="EfficientNet")
  optimizer = Adam(learning_rate=LEARNING_RATE)
  model.compile(
      optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
  )
  return model

## Training

In [72]:
# Obtenemos los generadores para training y validaton
train_gen = DataGenerator(folder=TRAIN_FOLDER, dic_images=dic_images, batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=True, predict=False)
valid_gen = DataGenerator(folder=VALID_FOLDER, dic_images=dic_images, batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=False, predict=False)

In [76]:
# Construimos el modelo (tanto el modelo base como el nuestro)
model = build_model()

In [77]:
# Entrenamos la red
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=EPOCHS,
)

Epoch 1/20
1002/1002 [==============================] - ETA: 0s - loss: 0.4986 - accuracy: 0.7646

ValueError: ignored